# Евклидова задача коммивояжёра

# Новый раздел

In [1]:
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

from time import time

In [2]:
import cupy as cp

In [3]:
def swapMutate(population,  n):
    """
    функция высчитывания мутаций у популяции:
    меняет два рандомных элемента у каких-то 
    представителей популяции
    population - соответственно популяция
    n - количество мутаций в популяции
    """
    
    indices = sps.randint(0, population.shape[0]).rvs(size=n)
    mutations = cp.copy(population[indices])
    for i in range(n):
        index1, index2 = sps.randint(0,population.shape[1]).rvs(2)
        mutations[i, index1], mutations[i, index2] = \
        mutations[i, index2], mutations[i, index1]
    return mutations   


def invMutate(population, n):
    """
    Inversion Mutation
    """
    indices = sps.randint(0, population.shape[0]).rvs(size=n)
    mutations = cp.copy(population[indices])
    for i in range(n):
        index1, index2 = sps.randint(1,population.shape[1]).rvs(2)
        mutations[i, index1: index2+1] = mutations[i, index2: index1-1:-1]
    return mutations    

In [4]:
def PMX(specimen1, specimen2):
    """
    Partitially-Mapped Crossover
    """
    index1, index2 = cp.sort(sps.randint(0, len(specimen1)).rvs(size=2))
    offspring = cp.ones(len(specimen1))*-1
    offspring[index1 : index2 + 1] = specimen1[index1 : index2 + 1]
    k = 0
    for el in specimen2:
        if (k == index1):
            k = index2 + 1
        if (el not in specimen1[index1 : index2 + 1]):
            offspring[k] = el
            k += 1
    return offspring


def CX(specimen1, specimen2):
    """
    Cycle Crossover
    """
    argparent1 = cp.argsort(specimen1)
    argparent2 = cp.argsort(specimen2) #индексы отсорт. массива 2
    
    offspring = cp.ones(len(specimen1))*-1
    offspring = offspring.astype(cupy.int)
    offspring[0] = specimen1[0]
    ind = 0
    curr_argp = argparent2
    curr_spec = specimen1
    for i in range(len(offspring)-1):
        ind = curr_argp[offspring[ind]]
        if (offspring[ind] != -1):
            ind  = cp.argmin(offspring)
            if (cp.all(curr_spec==specimen1)):
                curr_argp = argparent1
                curr_spec = specimen2
            else:
                curr_argp = argparent2
                curr_spec = specimen1
        offspring[ind] = curr_spec[ind]        
    return offspring


#само скрещивание популяции   
def getCrossover(population, func):
    """
    скрещивание популяции - смена поколения
    population - популяция
    func - функция скрещивания
    """
    m = population.shape[0] * (population.shape[0]+1) / 2
    generation = cp.zeros((int(m), population.shape[1]))
    k = 0
    for i in range(population.shape[0]):
        for j in range(i, population.shape[0]):
            offspring = func(population[i], population[j])
            generation[k] = offspring
            k += 1
    return generation

In [5]:
def createPopulation(Graph, n):
    """
    создание популяции путем
    рандомного перемешивания 
    
    n - размер популяции
    """
    specimen = cp.arange(Graph.shape[0]) 
    population = cp.zeros((n, Graph.shape[0]))
    for i in range(n):
        arr = cp.copy(specimen)
        cp.random.shuffle(arr)
        population[i] = arr
    return population

#длина пути для какого-то представителя популяции
def pathLength(Graph, specimen):
    """
    считает длину цикла, соответствующего 
    массиву индексов specimen
    последнее, замыкающее ребро цикла -
    считается между последним и 1ым эл-ами массива
    
    specimen - массив индексов вершин графа - особь популяции
    """
    length = 0
    for i in range(Graph.shape[0]):
        length += cp.linalg.norm(Graph[int(specimen[i])] - \
            Graph[int(specimen[(i + 1) % Graph.shape[0]])])
    return length


def sortPopulation(Graph, population):
    """
    функция отбора - сортировка популяции  по длине цикла
    """
    #key = lambda x:pathLength(Graph, x)
    #lengths = [pathLength(Graph, population[i]) for i in range(len(population))]
    #indexes = cupy.argsort(lengths)
    #population = population[indexes]
    return cp.array(sorted(population, key= lambda x:pathLength(Graph, x)))
    

#сам генетический алгоритм    
def Genetic(Graph, iter_count, popul_count, mutate_count=-1, mutate_func=swapMutate, cross_func=PMX):
    """
    Генетический алогритм
    Graph - граф в виде массива точек на плоскости
    iter_count - количество итераций
    popul_count - размер популяции
    mutate_count - количество мутаций
    mutate_func - функция мутации
    cross_func - функция скрещивания
    выводит массив индексов - "лучшую" особь популяции
    """
    population = createPopulation(Graph, popul_count)
    
    if (mutate_count == -1):
        mutate_count = int(popul_count / 4)
    
    for i in tqdm(range(iter_count)):
        mutations = mutate_func(population, mutate_count)
        offsprings = getCrossover(population, cross_func)

        population = cp.vstack((population, mutations, offsprings))
        population = sortPopulation(Graph, population)[:popul_count]

    return population[0]       

In [6]:
def getRandomGraph(N):
    """
    генерирует произвольный граф размера N
    каждая координата генерируется из равномерного
    распеделения на отрезке [0, N]
    """
    Graph = list()

    for i in range(N):
        Graph.append([sps.uniform(0, N).rvs(), sps.uniform(0, N).rvs()])
    return cp.array(Graph)


def getCycleIndices(indices):
    """
    возвращает "зацикленный" массив индексов -
    для удобства вывода на график
    """
    new_indices = cp.ones(len(indices) + 1)
    new_indices[:len(indices)] = indices
    new_indices[-1] = indices[0]
    return new_indices.get().astype(int)

In [9]:
graph_sizes = map(int, np.linspace(10, 400, 20))
iter_count = 100 # количество итераций
popul_count = 20 #  размеры популяций
times_cupy = np.zeros(20)

for k, graph_size in enumerate(graph_sizes):
    Graph = getRandomGraph(graph_size)
    time_start = time()
    indexes = getCycleIndices(Genetic(Graph, iter_count, popul_count))
    time_end = time()
    times_cupy[k] = time_end - time_start

  8%|▊         | 8/100 [05:01<57:58, 37.81s/it]

KeyboardInterrupt: ignored

In [11]:
np.save('times_cupy.npy', times_cupy)